## Notebook Imports

In [1]:
import pandas as pd
import numpy as np

## Constants

In [2]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/02_Training/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/02_Training/test-target.txt'

VOCAB_SIZE = 2500

## Read and Load Features from .txt Files into Numpy Arrays

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2058,    0,    1],
       [5795, 2060,    0,    2],
       [5795, 2283,    0,    1],
       [5795, 2364,    0,    1],
       [5795, 2433,    0,    1]])

In [7]:
print('Nr of rows in training file', sparse_train_data.shape[0])
print('Nr of rows in testing file', sparse_test_data.shape[0])

Nr of rows in training file 258320
Nr of rows in testing file 117613


In [8]:
print('Nr of email in training file', np.unique(sparse_train_data[:, 0]).size)
print('Nr of email in testing file', np.unique(sparse_test_data[:, 0]).size)

Nr of email in training file 4015
Nr of email in testing file 1724


## How to Create a Empty DataFrame

In [9]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [10]:
len(column_names)

2502

In [11]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [12]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [13]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
sparse_train_data[3:10]

array([[ 0,  7,  1,  3],
       [ 0, 11,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 18,  1,  2],
       [ 0, 19,  1,  2],
       [ 0, 23,  1,  1],
       [ 0, 26,  1,  2]])

In [15]:
sparse_train_data[10][3]

1

## Create a Full Matrix from a Sparse Matrix

In [16]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas DataFrame.
    Keyword arguments:
    sparse_matrix -- numpy array.
    nr_word -- size of vacabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column.
    word_idx -- position of the word id in the sparse matrix. Default: 2st column.
    cat_idx -- position of the label (spam is 1, non-spam is 0). Default: 3st column.
    freq_idx -- position of the occurrence of words in the sparse matrix. Default: 1st column.
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    index_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=index_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
        
    return full_matrix

In [17]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 11.2 s


In [18]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Naive Bayes Model
### Calculating the Probability of Spam

In [19]:
full_train_data.CATEGORY.size

4015

In [20]:
full_train_data.CATEGORY.sum()

1250

In [21]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('Probability of spam is ', prob_spam)

Probability of spam is  0.31133250311332505


### Total Number of Words/Tokens

In [22]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.head()

DOC_ID
0     87
1     53
2     40
3    182
4     43
dtype: int64

In [24]:
total_wc = email_lengths.sum()
total_wc

431024

### Number of Tokens in Spam & Ham Emails

In [25]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1250,)

In [26]:
spam_wc = spam_lengths.sum()
spam_wc

178692

In [27]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [28]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252332

In [29]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [30]:
total_wc - spam_wc - nonspam_wc

0

In [31]:
print('Average nr of words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average nr of words in ham emails {:.3f}'.format(nonspam_wc / ham_lengths.shape[0]))

Average nr of words in spam emails 143
Average nr of words in ham emails 91.259


### Suming the Tokens occuring in Spam

In [32]:
full_train_features.shape

(4015, 2500)

In [33]:
train_spam_tokens = full_train_features[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_spam_tokens.shape

(1250, 2500)

In [36]:
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_spam_tokens.shape

(2500,)

In [37]:
summed_spam_tokens.tail()

2495    29
2496     1
2497    22
2498    10
2499    18
dtype: int64

### Summing the Toekns Occuring in Ham

In [38]:
train_ham_tokens = full_train_features[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis = 0) + 1

In [39]:
summed_ham_tokens.shape

(2500,)

In [40]:
summed_ham_tokens.tail()

2495    16
2496    36
2497     1
2498    24
2499    20
dtype: int64

In [41]:
train_ham_tokens[2499].sum() + 1

20

### P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [42]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012015
1    0.005166
2    0.006733
3    0.011182
4    0.006684
dtype: float64

In [43]:
prob_tokens_spam.sum()

1.0

### P(Token | Ham) - Probability that a Token Occurs given the Email is Nonspam

In [44]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

### P(Token) - Probability that Token Occurs

In [45]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0

### Save the Trained Model

In [46]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data

In [47]:
sparse_test_data.shape

(117613, 4)

In [48]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 5.42 s


In [49]:
x_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [50]:
np.savetxt(TEST_FEATURE_MATRIX, x_test)
np.savetxt(TEST_TARGET_FILE, y_test)